### 라이브러리

In [48]:
import pandas as pd
import os
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
from gensim.summarization.summarizer import summarize
from newspaper import Article

### 데이터 전처리

In [ ]:
# # 파일에 있는 모든 csv 파일 불러오기

# # 불러온 파일이 있는 경로
# path = './부산_장애인_교통.csv'
# file_list = os.listdir(path)
# file_list_py = [file for file in file_list if file.endswith('.csv')] # 파일명이 끝이 .csv 인 경우

In [ ]:
# # csv 파일들을 DataFrame으로 불러와서 concat

# news = pd.DataFrame()
# for i in file_list_py:
#     data = pd.read_csv(path+'/'+i, encoding='utf-8')
#     news = pd.concat([news, data])
    
#     news = news.reset_index(drop=True)
# news

In [49]:
news = pd.read_csv('./data/저상버스 불편.csv', encoding='utf-8')
news

,title,content
0,대전·세종 BRT 도로에 2층 전기저상버스 달린다,[\n다음 달 14일부터 B1 노선에 2대 운행…하루 20회\n\n\n\n대전시가 ...
1,"대전시, 2층 전기 저상버스 내달부터 운행",[\n장애인-교통약자 등 이용 가능\n\n\n\n대전시가 중부권 최초로 다음 달 1...
2,[무등의 아침] 11일째 멈춰 선 목포 시내버스…보조금에만 목매는 회사,"[\n- ""목포 시내버스 회사 노조, 18일 파업 돌입..11일째 이어져""- ""노조..."
3,‘버스 운행 방해’ 박경석 전장연 대표 1심 집행유예,"[\n버스 앞문, 몸 쇠사슬로 연결…23분간 운행 방해1심 징역 4월·집행유예 2년..."
4,"法 “버스·지하철 운행 방해, 정당화 안 돼” 전장연 대표 집행유예",[\n징역 4개월·집유 2년 선고“집회 방식 재고 당부했음에도…”박 대표 “선생님 ...
...,...,...
1267,"""가짜로 살고 싶지 않아 '나쁜 장애인' 됐습니다"" [청년정치 와글와글]",[\n[청년정치와글와글-인터뷰] 변재원 전국장애인차별철폐연대 정책국장 [이진순] ...
1268,"""장애인정책 확대하라"" 장애인단체 '도로점거'…청주도심 한때 정체",[\n장애인 관련 정책 확대·지사 면담 요구 3시간 점거이시종 지사 면담 약속 공문...
1269,"낮은 문턱, 계단 사라진 놀이터... '무장애 도시' 아시나요",[\n자체 조례 제정 등 장애물 없는 생활환경 실현 나선 지역들 [월간 옥이네] ...
1270,"'우당 이회영' 기념관 완공, 오세훈의 '남산 르네상스' 마무리",[\n1920년 독립군의 화력 뒷받침한 '체코 군단' 무기도 특별전시[손병관 기자]...


In [50]:
# news = news.drop(columns=['Unnamed: 0','날짜','기사제목'], axis=0)
# news = news.drop(columns=['Unnamed: 0','날짜'], axis=0)
news = news.drop(columns=['Unnamed: 0','기사제목'], axis=0)
news["기사내용"] = news["기사내용"].str.replace("\t", "").str.replace(".", "\n")
news

C:\Users\admin\AppData\Local\Temp\ipykernel_11760\2198213908.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  news["content"] = news["content"].str.replace("\t", "").str.replace(".", "\n")


,content
0,[\n다음 달 14일부터 B1 노선에 2대 운행…하루 20회\n\n\n\n대전시가 ...
1,[\n장애인-교통약자 등 이용 가능\n\n\n\n대전시가 중부권 최초로 다음 달 1...
2,"[\n- ""목포 시내버스 회사 노조, 18일 파업 돌입\n\n11일째 이어져""- ""..."
3,"[\n버스 앞문, 몸 쇠사슬로 연결…23분간 운행 방해1심 징역 4월·집행유예 2년..."
4,[\n징역 4개월·집유 2년 선고“집회 방식 재고 당부했음에도…”박 대표 “선생님 ...
...,...
1267,[\n[청년정치와글와글-인터뷰] 변재원 전국장애인차별철폐연대 정책국장 [이진순] ...
1268,[\n장애인 관련 정책 확대·지사 면담 요구 3시간 점거이시종 지사 면담 약속 공문...
1269,[\n자체 조례 제정 등 장애물 없는 생활환경 실현 나선 지역들 [월간 옥이네] ...
1270,[\n1920년 독립군의 화력 뒷받침한 '체코 군단' 무기도 특별전시[손병관 기자]...


In [51]:
# 요약 글 데이터프레임으로 만들어주기
for i in range(0, len(news)):
    try:
        text = news['기사내용'][i]
        # 문자열로 만들어주기
        text = (''.join(summarize(text, ratio=0.2).split('\n')))
        # print(text_list)
    except Exception as e:
        print(i,",", end='')
        continue

96 ,

In [52]:
news = news.drop([96])
news = news.reset_index(drop=True)
news

,content
0,[\n다음 달 14일부터 B1 노선에 2대 운행…하루 20회\n\n\n\n대전시가 ...
1,[\n장애인-교통약자 등 이용 가능\n\n\n\n대전시가 중부권 최초로 다음 달 1...
2,"[\n- ""목포 시내버스 회사 노조, 18일 파업 돌입\n\n11일째 이어져""- ""..."
3,"[\n버스 앞문, 몸 쇠사슬로 연결…23분간 운행 방해1심 징역 4월·집행유예 2년..."
4,[\n징역 4개월·집유 2년 선고“집회 방식 재고 당부했음에도…”박 대표 “선생님 ...
...,...
1266,[\n[청년정치와글와글-인터뷰] 변재원 전국장애인차별철폐연대 정책국장 [이진순] ...
1267,[\n장애인 관련 정책 확대·지사 면담 요구 3시간 점거이시종 지사 면담 약속 공문...
1268,[\n자체 조례 제정 등 장애물 없는 생활환경 실현 나선 지역들 [월간 옥이네] ...
1269,[\n1920년 독립군의 화력 뒷받침한 '체코 군단' 무기도 특별전시[손병관 기자]...


In [53]:
# 리스트로 담기
sum_list = []

for i in range(0, len(news)):
    text = news['기사내용'][i]
    text_list = (''.join(summarize(text, ratio=0.2).split('\n')))
    
    sum_list.append(text_list)

In [54]:
total = pd.DataFrame(sum_list, columns=['기사요약'])
total

,기사요약
0,대전시가 도입하는 2층 전기 저상버스[양영석 기자](대전=연합뉴스) 양영석 기자 =...
1,대전시가 중부권 최초로 다음 달 14일부터 운행하는 2층 전기 저상버스대전시 제공대...
2,5 MHz ■ 진행 : 정길훈 앵커(전 보도국장) ■ 출연 : 김대영 리포터 ■ ...
3,박경석 장애인차별철폐연대 공동대표[연합][헤럴드경제=유동현 기자] 장애인 이동권 보...
4,연합뉴스장애인 이동권 보장을 요구하며 퇴근길 버스 운행을 방해한 혐의로 기소된 박경...
...,...
1266,[청년정치와글와글-인터뷰] 변재원 전국장애인차별철폐연대 정책국장 [이진순] ...
1267,충북장애인차별철폐연대 회원들이 13일 충북도청 서문 앞 도로를 점거하고 충북도의 장...
1268,자체 조례 제정 등 장애물 없는 생활환경 실현 나선 지역들 [월간 옥이네] ▲ ...
1269,ⓒ 서울시 제공 조선총독부 관저와 중앙정보부 등이 자리잡았던 남산 예장자락이 시...


In [55]:
total.to_csv('./res/저상버스_불편_요약.csv')